In [1]:
import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image

import code
import os
#import tensorflow.python.platform

import numpy as np
import tensorflow.keras as keras
from keras import layers, models, datasets,regularizers, Model, Input
from keras.layers import concatenate, Conv2D, Conv2DTranspose, BatchNormalization, MaxPooling2D, Dropout
from helpers_images import * 


In [2]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [3]:
NUMBER_TRAINING_EXAMPLES = 100
NUMBER_TEST_EXAMPLES = 50
DATA_LOADED = False

In [4]:
training_data_directory = "../data/training/"
test_data_directory = "../data/test_set_images/"
if not(DATA_LOADED):    
    train_data_orig = extract_train_data(NUMBER_TRAINING_EXAMPLES)
    train_labels_orig =  extract_labels(training_data_directory, NUMBER_TRAINING_EXAMPLES,True)
    DATA_LOADED = True

print(train_data_orig.shape)
print(train_labels_orig.shape)

Loading ../data/training/images/satImage_001.png
Loading ../data/training/images/satImage_002.png
Loading ../data/training/images/satImage_003.png
Loading ../data/training/images/satImage_004.png
Loading ../data/training/images/satImage_005.png
Loading ../data/training/images/satImage_006.png
Loading ../data/training/images/satImage_007.png
File ../data/training/images/satImage_008.png does not exist
Loading ../data/training/images/satImage_009.png
Loading ../data/training/images/satImage_010.png
File ../data/training/images/satImage_011.png does not exist
Loading ../data/training/images/satImage_012.png
Loading ../data/training/images/satImage_013.png
Loading ../data/training/images/satImage_014.png
Loading ../data/training/images/satImage_015.png
Loading ../data/training/images/satImage_016.png
Loading ../data/training/images/satImage_017.png
Loading ../data/training/images/satImage_018.png
Loading ../data/training/images/satImage_019.png
Loading ../data/training/images/satImage_020.

Loading ../data/training/groundtruth/satImage_067.png
Loading ../data/training/groundtruth/satImage_068.png
Loading ../data/training/groundtruth/satImage_069.png
Loading ../data/training/groundtruth/satImage_070.png
Loading ../data/training/groundtruth/satImage_071.png
Loading ../data/training/groundtruth/satImage_072.png
Loading ../data/training/groundtruth/satImage_073.png
Loading ../data/training/groundtruth/satImage_074.png
Loading ../data/training/groundtruth/satImage_075.png
Loading ../data/training/groundtruth/satImage_076.png
Loading ../data/training/groundtruth/satImage_077.png
File ../data/training/groundtruth/satImage_078.png does not exist
Loading ../data/training/groundtruth/satImage_079.png
Loading ../data/training/groundtruth/satImage_080.png
Loading ../data/training/groundtruth/satImage_081.png
Loading ../data/training/groundtruth/satImage_082.png
Loading ../data/training/groundtruth/satImage_083.png
Loading ../data/training/groundtruth/satImage_084.png
Loading ../data/

In [5]:
#plt.imshow(train_data_orig[187])

In [6]:
IMG_PATCH_SIZE = 400
train_data = train_data_orig
train_labels = train_labels_orig

In [7]:
def encoding_block(input_, nbr_filters = 16, pooling_ = True, dropout = True):
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (input_)
    conv = BatchNormalization()(conv)
    if dropout:
        conv = Dropout(0.2) (conv)
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (conv)
    conv = BatchNormalization()(conv)
    if pooling_:
        pooling = MaxPooling2D((2,2))(conv)
        return conv, pooling
    return conv

In [8]:
def decoding_block(conv, previous_conv,nbr_filters = 16, dropout = True):
    upsample = Conv2DTranspose(nbr_filters, (2, 2),kernel_regularizer=regularizers.l2(0.01), strides=(2, 2), padding='same') (conv)
    upsample = concatenate([upsample, previous_conv])
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (upsample)
    conv = BatchNormalization() (conv)
    if dropout:
        conv = Dropout(0.2) (conv)
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (conv)
    conv = BatchNormalization() (conv)
    return conv

In [9]:
def create_custom_unet(nbr_filters = 16, dropout = True):
    inputs = Input((IMG_PATCH_SIZE, IMG_PATCH_SIZE, 3))
    conv1, pooling1 = encoding_block(inputs, nbr_filters, True, dropout)
    conv2, pooling2 = encoding_block(pooling1, nbr_filters*2, True,dropout)
    conv3, pooling3 = encoding_block(pooling2, nbr_filters*4, True,dropout)
    conv4, pooling4 = encoding_block(pooling3, nbr_filters*8, True,dropout)
    conv5 = encoding_block(pooling4, nbr_filters*16,False, dropout)

    conv6 = decoding_block(conv5, conv4, nbr_filters*8, dropout)
    conv7 = decoding_block(conv6, conv3, nbr_filters*4, dropout)
    conv8 = decoding_block(conv7, conv2, nbr_filters*2, dropout)
    conv9 = decoding_block(conv8, conv1, nbr_filters, dropout)
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model


In [10]:
from keras import backend as K

def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [11]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystopper = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 10,
                          verbose = 1,
                          mode = "min",
                          restore_best_weights = True)
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               mode="min",
                               min_delta=1e-4)

In [12]:
model = create_custom_unet(16, True)
model.summary()
model.compile(optimizer='adam',
            loss=soft_dice_loss,
          metrics=[dice_coef])       

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400, 400, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 400, 400, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 400, 400, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [13]:
epochs = 200
history = model.fit(train_data, train_labels, epochs=epochs, validation_split = 0.05, batch_size = 4,callbacks = [ earlystopper, lr_reducer])

Epoch 1/200
219/219 [==============================] - 16s 49ms/step - loss: 0.9371 - dice_coef: 0.5781 - val_loss: 0.5102 - val_dice_coef: 0.6175 - lr: 0.0010
Epoch 2/200
219/219 [==============================] - 9s 41ms/step - loss: 0.3983 - dice_coef: 0.7211 - val_loss: 0.3367 - val_dice_coef: 0.7728 - lr: 0.0010
Epoch 3/200
219/219 [==============================] - 9s 41ms/step - loss: 0.3409 - dice_coef: 0.7647 - val_loss: 0.3048 - val_dice_coef: 0.7935 - lr: 0.0010
Epoch 4/200
219/219 [==============================] - 9s 41ms/step - loss: 0.3043 - dice_coef: 0.7914 - val_loss: 0.2568 - val_dice_coef: 0.8345 - lr: 0.0010
Epoch 5/200
219/219 [==============================] - 9s 41ms/step - loss: 0.2917 - dice_coef: 0.7998 - val_loss: 0.2747 - val_dice_coef: 0.8099 - lr: 0.0010
Epoch 6/200
219/219 [==============================] - 9s 41ms/step - loss: 0.2734 - dice_coef: 0.8141 - val_loss: 0.4924 - val_dice_coef: 0.5888 - lr: 0.0010
Epoch 7/200
219/219 [========================

219/219 [==============================] - 9s 41ms/step - loss: 0.1188 - dice_coef: 0.9194 - val_loss: 0.0976 - val_dice_coef: 0.9400 - lr: 1.0000e-04
Epoch 51/200
219/219 [==============================] - 9s 41ms/step - loss: 0.1190 - dice_coef: 0.9189 - val_loss: 0.0954 - val_dice_coef: 0.9420 - lr: 1.0000e-04
Epoch 52/200
219/219 [==============================] - 9s 42ms/step - loss: 0.1187 - dice_coef: 0.9190 - val_loss: 0.0975 - val_dice_coef: 0.9399 - lr: 1.0000e-04
Epoch 53/200
219/219 [==============================] - 9s 42ms/step - loss: 0.1165 - dice_coef: 0.9208 - val_loss: 0.0940 - val_dice_coef: 0.9428 - lr: 1.0000e-04
Epoch 54/200
219/219 [==============================] - 9s 42ms/step - loss: 0.1159 - dice_coef: 0.9209 - val_loss: 0.0929 - val_dice_coef: 0.9436 - lr: 1.0000e-04
Epoch 55/200
219/219 [==============================] - 9s 42ms/step - loss: 0.1156 - dice_coef: 0.9212 - val_loss: 0.0930 - val_dice_coef: 0.9431 - lr: 1.0000e-04
Epoch 56/200
219/219 [=======

219/219 [==============================] - 9s 41ms/step - loss: 0.0943 - dice_coef: 0.9348 - val_loss: 0.0781 - val_dice_coef: 0.9506 - lr: 1.0000e-05
Epoch 100/200
219/219 [==============================] - 9s 41ms/step - loss: 0.0940 - dice_coef: 0.9349 - val_loss: 0.0782 - val_dice_coef: 0.9505 - lr: 1.0000e-05
Epoch 101/200
219/219 [==============================] - ETA: 0s - loss: 0.0942 - dice_coef: 0.9347
Epoch 00101: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
219/219 [==============================] - 9s 41ms/step - loss: 0.0942 - dice_coef: 0.9347 - val_loss: 0.0779 - val_dice_coef: 0.9506 - lr: 1.0000e-05
Epoch 102/200
219/219 [==============================] - 9s 42ms/step - loss: 0.0935 - dice_coef: 0.9355 - val_loss: 0.0777 - val_dice_coef: 0.9509 - lr: 1.0000e-06
Epoch 103/200
219/219 [==============================] - 9s 41ms/step - loss: 0.0939 - dice_coef: 0.9351 - val_loss: 0.0776 - val_dice_coef: 0.9510 - lr: 1.0000e-06
Epoch 104/200
219/219 

In [14]:
import pathlib
directory_path = pathlib.Path(test_data_directory)

In [15]:
all_images = []
for i in range(1,51):
    image_in_directory = list(directory_path.glob("test_" + str(i) + "/*"))
    all_images.extend(image_in_directory)
img_height = img_width = 608

In [16]:
all_images_arrays = []
for i in range(len(all_images)):
    img = tf.keras.utils.load_img(
        all_images[i], target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    all_images_arrays.append(img_array)
all_images_arrays = numpy.array(all_images_arrays)/255.0

In [17]:
predictions_testing = "./predictions/"
if not os.path.isdir(predictions_testing):
    os.mkdir(predictions_testing)

In [18]:
def predict(k):
    img = all_images_arrays[k]
    extract_1 = img[:400, :400]
    extract_2 = img[:400, -400:]
    extract_3 = img[-400:, :400]
    extract_4 = img[-400:, -400:]
    output_prediction = []
    output_prediction.append(model(np.expand_dims(extract_1 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_2 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_3 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_4 , axis=0)).numpy() )

    output_prediction[0] = output_prediction[0].reshape(400,400)
    output_prediction[1] = output_prediction[1].reshape(400,400)
    output_prediction[2] = output_prediction[2].reshape(400,400)
    output_prediction[3] = output_prediction[3].reshape(400,400)
    gt_img = numpy.zeros([img.shape[0], img.shape[1]])
    gt_img[:400,:400] = output_prediction[0]
    gt_img[:400, -400:] = output_prediction[1]
    gt_img[-400: , :400] = output_prediction[2]
    gt_img[-400: , -400:] = output_prediction[3]
    oimg = make_img_overlay(img, gt_img)
    gt_img8 = img_float_to_uint8(gt_img)          
    gt_img_3c = numpy.zeros((img.shape[0], img.shape[1], 3), dtype=numpy.uint8)
    gt_img_3c[:, :, 0] = gt_img8
    gt_img_3c[:, :, 1] = gt_img8
    gt_img_3c[:, :, 2] = gt_img8
    overlay = make_img_overlay(img, gt_img)
    Image.fromarray(gt_img_3c).save(predictions_testing + "prediction_image_" + str(k+1) + ".png")
    oimg.save(predictions_testing  + "overlay_" + str(k) + ".png")

In [19]:
for i in range(len(all_images_arrays)):
    predict(i)

In [20]:
5

5

In [21]:
model.save("./models/model_unet_idk_0.2dropout_0.05validation_0.0001l2_12batchsize.h5")